<a href="https://colab.research.google.com/github/zackives/upenn-cis5450-hw/blob/main/Module_1_Data_Acquisition_Wrangling_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I of Big Data Analytics - Notebook 2

## Recall

As we start our journey into Big Data Analytics, the first thing we need to do is **get the data** in the form we need for analysis!  We'll start with an overview of how to acquire and *wrangle* data.

This notebook will be built incrementally to consider several tasks:

* Acquiring data from files and remote sources
* Information extraction over HTML content
* A basic "vocabulary" of operators over tables (the relational algebra)
* Basic manipulation using SQL in DuckDB

* "Data wrangling" or integration:
  * Cleaning and filtering data, using rules and based operations
  * Linking data across dataframes or relations
  * The need for approximate match and record linking
  * Different techniques

## Before you start this notebook

Make sure you did the exercises in [Notebook 1](https://colab.research.google.com/github/zackives/cis5450-hw/blob/main/Module_1_Data_Acquisition.ipynb)!

## The Motivating Question
To illustrate the principles, we focus on the question of **how old company CEOs and founders** (in general, leaders) are.  The question was in part motivated by the following New York Times article:

* Founders of Successful Tech Companies Are Mostly Middle-Aged: https://www.nytimes.com/2019/08/29/business/tech-start-up-founders-nest.html?searchResultPosition=2

So let's test this hypothesis!

## Initial Libraries

We'll be using [DuckDB](https://duckdb.org/) as a means of managing our tables.  DuckDB works like a Python library, but manages a full SQL database (in files).  It also integrates very nicely with Pandas, so we'll use it in this course.

In [ ]:
!pip3 install duckdb

In [ ]:
!pip3 install lxml

In [ ]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

In [ ]:
!pip3 install penngrader-client

For quiz credit you'll need to update your student ID here!

In [ ]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 99999999 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

Quizzes will cumulatively count as HW9... Don't edit this...

In [ ]:
%set_env HW_ID=cis5450_25f_HW9

In [ ]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

In [ ]:
# Imports we'll use through the notebook, collected here for simplicity

# For parsing dates and being able to compare
import datetime

# For fetching remote data
import urllib
import urllib.request

# Pandas dataframes and operations
import pandas as pd

# Numpy matrix and array operations
import numpy as np

# Sqlite is a simplistic database
import duckdb

# Data visualization
import matplotlib

from lxml import etree

# 1. Reload from HW1

Here is a quick reload of the data from Homework 1.

For simplicity, we reload all of the following:
* `company_ceos_df` -- Wikipedia list of companies and their CEOs
* `exec_df` -- crawled executive names and birthdays
* `company_data_df` -- 7M entries about companies
* `company_info_df` -- CSV about companies and lines of business

In [ ]:
!wget -nc https://storage.googleapis.com/penn-cis5450/companies_sorted.csv

Here's a helper for importing HTML

In [ ]:
import requests
import pandas as pd
from io import StringIO

headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def import_html(url):
  # Let's read an HTML table!
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
  }
  page = requests.get(url, headers=headers).text
  return page


In [ ]:
from io import StringIO

company_ceos_df = pd.read_html(StringIO(import_html('https://en.wikipedia.org/wiki/List_of_chief_executive_officers#List_of_CEOs')))[1]

pages = []
page_urls = []

for i, executive in enumerate(company_ceos_df['Executive']):
  url = 'https://en.wikipedia.org/wiki/' + executive.replace(' ', '_')

  page = url.split("/")[-1] #extract the person name at the end of the url

  # An issue: some of the accent characters won't work.  We need to convert them
  # into an HTML URL.  We'll split the URL, then use "parse.quote" to change
  # the structure, then re-form the URL
  url_list = list(urllib.parse.urlsplit(url))
  url_list[2] = urllib.parse.quote(url_list[2])
  url_ascii = urllib.parse.urlunsplit(url_list)
  try:
    response = import_html(url_ascii)
    pages.append(response)
    page_urls.append(url)
  except requests.exceptions.RequestException as e:
    print(e)


# Use lxml.etree.HTML(...) on the HTML content of each page to get a DOM tree that
# can be processed via XPath to extract the bday information.  Store the CEO name,
# webpage, and the birthdate (born) in exec_df.

# We first check that the HTML content has a table of type `vcard`,
# and then extract the `bday` information.  If there is no birthdate, the datetime
# value is NaT (not a type).

rows = []
for i, page in enumerate(pages):
  url = page_urls[i]

  tree = etree.HTML(page)  #create a DOM tree of the page
  bday = tree.xpath('//table[contains(@class,"vcard")]//span[@class="bday"]/text()')
  if len(bday) > 0:
      name = url[url.rfind('/')+1:] # The part of the URL after the last /
      rows.append({'name': name, 'page': url,
                  'born': datetime.datetime.strptime(bday[0], '%Y-%m-%d')})
  else:
          rows.append({'name': url[url.rfind('/')+1:], 'page': url
                                    , 'born': np.datetime64('NaT')})

exec_df = pd.DataFrame(rows)
exec_df['clean_name'] = exec_df['name'].apply(lambda x: x.replace('_', ' '))
company_data_df = pd.read_csv('companies_sorted.csv')
countries_df = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")

data = urllib.request.urlopen(\
       'https://gist.github.com/jvilledieu/c3afe5bc21da28880a30/raw/a344034b82a11433ba6f149afa47e57567d4a18f/Companies.csv')

company_info_df = pd.read_csv(data)


Now let's use DuckDB to allow working with on-disk and in memory data.

In [ ]:
con = duckdb.connect('local.db')
con.sql("""CREATE TABLE IF NOT EXISTS company_data AS
           SELECT *
           FROM company_data_df""")
con.sql("""create table if not exists company_ceos as select * from company_ceos_df""")
con.sql("""create table if not exists executives as select * from exec_df""")

# query the table
con.table("company_data").show()

# 2.0 Data Transformation and Querying

Looking at our data to clean via *projection*...

Generally, we can extract one "narrower" table form another by using **double brackets**.

In [ ]:
# Let's take a look at the data.  Here's a way of PROJECTING the exec_df dataframe into
# a smaller table

exec_df[['name', 'born']]

In [ ]:
# In SQL it's SELECT with the fields FROM the table
con.sql('select name, born from executives')

In [ ]:
# If I use single brackets, I can extract a single column as a Series.
exec_df['name']

In [ ]:
# We can use SQL over the dataframe OR here
duckdb.sql('SELECT name FROM exec_df')

In [ ]:
# Notice anything awry?

for person in exec_df['name']:
    print (person)

In [ ]:
def to_space(x):
  return x.replace('_', ' ')

# Let's use *apply* to call a function over each element, returning a new Series
exec_df['name'].apply(to_space)

In [ ]:
# Let's use *apply* to call a function over each element, returning a new Series
exec_df['name'].apply(lambda x: x.replace('_', ' '))

In [ ]:
# I can also use *apply* to call a function over the rows of a dataframe
exec_df.apply(lambda x: x['name'].replace('_', ' '), axis=1)

In [ ]:
# Let's clean the name by removing underscores...
exec_df['clean_name'] = exec_df['name'].apply(lambda x: x.replace('_', ' '))

exec_df

In [ ]:
exec_df.rename(columns={'name': 'old_name'})

In [ ]:
# We can do the same via SQL.

duckdb.sql("""SELECT name, replace(name, '_', ' ') AS clean_name
              FROM exec_df""")

## 2.1. Selecting a subset of the rows

In [ ]:
# Here's a column

exec_df['clean_name']

In [ ]:
# We can apply a test (predicate) to each column, returning a Series of boolean true/false values

exec_df['clean_name'] == 'Julie Sweet'

In [ ]:
# If we compose these, we'll get only those rows where the boolean condition was True

exec_df[exec_df['clean_name'] == 'Julie Sweet']

SQL lets us use any case, but convention is to capitalize the SQL keywords such as `SELECT`, `FROM`, `WHERE` to aid in readability.  Also, we should use single-quotes for SQL strings, so we'll typically pass the SQL command in with double-quotes.

In [ ]:
duckdb.sql("SELECT * FROM exec_df WHERE clean_name='Julie Sweet'")

In [ ]:
exec_df[exec_df['clean_name'] == 'Julie Sweet'][['page']]


Here we'll use the triple-quote syntax for Python strings, which allows us to pass a multi-line string to SQL...


In [ ]:
duckdb.sql("""SELECT clean_name
            FROM exec_df
            WHERE clean_name='Julie Sweet'""")

In [ ]:
import numpy as np

exec_df.dropna(subset=['born'])

In [ ]:
duckdb.sql("""SELECT *
            FROM exec_df
            WHERE born IS NOT NULL""")

## 2.2. Joining Data

We start with a simple join between company_ceos_df and exec_df and persist it to the database.  We then check how many companies did not have a match on CEO name.

In [ ]:
exec_df[['clean_name', 'born']]

In [ ]:
# Remove any duplicate executive entries

exec_df = exec_df.drop_duplicates()

In [ ]:
company_ceos_df[['Executive', 'Company']]

In [ ]:
company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']],
                                                left_on=['Executive'],
                                                right_on=['clean_name'])

We can `JOIN ON` in the `FROM` clause.

In [ ]:
duckdb.sql("""
            SELECT Executive, Company, born
            FROM company_ceos_df JOIN exec_df ON Executive=clean_name
          """)

Note there is another way you'll sometimes see, in older versions of SQL... Which is to put the join as a `WHERE` condition:

In [ ]:
duckdb.sql("""
            SELECT Executive, Company, born
            FROM company_ceos_df, exec_df
            WHERE Executive=clean_name
          """)

OK, let's drop the cases where we don't have a CEO's birthday: these aren't useful!

In [ ]:
# Shall we skip the cases where we don't have the birthday?
duckdb.sql("""
            SELECT Executive, Company, born
            FROM company_ceos_df JOIN exec_df ON Executive=clean_name
            WHERE born is not null
          """)

In [ ]:
company_ceos_df[['Executive', 'Company']]

In [ ]:
exec_df[['clean_name', 'born']]

## 2.4. Finding the misses in the join with OUTER JOINs.

Note that the join above resulted in 174 rows.  However, there are more rows in company_ceos_df so we are missing some companies.  We can see which are missed using a LEFT OUTERJOIN (aka LEFT JOIN); setting "indicator= True" allows us to see which tuples in company_ceos_df failed to find a match (left_only, e.g. row 24 and 172).

In [ ]:
pd.set_option('display.max_rows', 200)
display(company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']],
                                                left_on=['Executive'],
                                                right_on=['clean_name'], how="left", indicator=True))



In [ ]:
pd.set_option('display.max_rows', 50)
result_df = company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']],
                                                left_on=['Executive'],
                                                right_on=['clean_name'], how="outer", indicator=True)

result_df[result_df['_merge'] != 'both']


We can also do this in SQL (there is no indicator but we can test for NULL):

In [ ]:
duckdb.sql("""
            SELECT Executive, Company, clean_name, born
            FROM company_ceos_df FULL JOIN exec_df ON Executive=clean_name
            WHERE clean_name is null or Company is null
          """)

## 2.3. Composing Joins

Of course, we can join the results of a join with another table -- representing a *composition*!



Let's join with company data!

In [ ]:
duckdb.sql("""SELECT Executive, Company, born
            FROM company_ceos_df
            JOIN exec_df ON Executive=replace(name, '_', ' ')
            JOIN company_data_df cd ON Company=cd.name
            WHERE born is not null""")

Hmm, what is wrong here?

Let's switch to the tables saved in DuckDB for these.

In [ ]:
con.sql('SELECT * from company_data')

Notice the case for `name`?

In [ ]:
con.sql("""SELECT Executive, Company, born
            FROM company_ceos
            JOIN executives ON Executive=replace(name, '_', ' ')
            JOIN company_data cd ON lower(Company)=lower(cd.name)
            WHERE born is not null
            ORDER BY Company""")

Hmm, there are duplicates!  This is because of fields in the `company_data` table that we don't care about. We can remove the duplicates via `SELECT DISTINCT`.

In [ ]:
con.sql("""SELECT DISTINCT Executive, Company, born
            FROM company_ceos
            JOIN executives ON Executive=replace(name, '_', ' ')
            JOIN company_data cd ON lower(Company)=lower(cd.name)
            WHERE born is not null
            ORDER BY Company""")

Can we do all of this in Pandas? Of course!

First, we need to lowercase the company names.

In [ ]:
company_ceos_df['company_lc'] = company_ceos_df['Company'].apply(lambda x: x.lower())

Notice this is slower than DuckDB?

In [ ]:
company_ceos_df.merge(exec_df.dropna(),
                      left_on=['Executive'],
                      right_on=['clean_name']).\
                      merge(company_data_df,
                            left_on='company_lc',
                            right_on='name')[['Executive','Company','born']].drop_duplicates().sort_values('Company')

## Section 2 Exercises

Hmm, there seem to be quite a few Roberts and Bobs. (Maybe also Roberta?)

Using either Pandas operations like `merge` and the various bracket notations (but **not** the `loc`/`iloc` commands if you know these): write a query that takes the `company_ceos_df` and returns all companies that are overseen by someone with names `Bob`, `Robert`, or `Roberta`.

First, define a function that *robustly* does this:

In [ ]:
def bob(name: str) -> bool:
  first_name = name # TODO: change this to get first name!

  return False # TODO: change this!

In [ ]:
from dill.source import getsource
grader.grade('bob_test', getsource(bob))

Now *select a subset of the rows* that have Bob/etc. using the function `bob` and the `apply` function.

In [ ]:
bob_co_df = None# TODO
bob_co_df

In [ ]:
grader.grade(test_case_id='bobs_your_uncle_or_aunt', answer=bob_co_df)